In [2]:
import cvxpy as cp
import numpy as np
import plotly.graph_objects as go
# Exports the plot to an interactive HTML file
import plotly.offline as pyo
import dash
from dash import dcc, html, Input, Output

In [3]:
# Define the data for a 3-stock portfolio
assets = ['A', 'B', 'C']
standard_deviations = np.array([20, 30, 50])  # Standard deviations (volatility) for each asset
esg_scores = np.array([10, 15, 20])  # ESG scores for each asset

# Initial portfolio allocation
initial_proportions = np.array([0.2, 0.3, 0.5])
initial_risk = np.sum(standard_deviations * initial_proportions)
initial_esg = np.sum(esg_scores * initial_proportions)

# Variables
proportions = cp.Variable(len(assets))

# Define constraints
constraints = [
    cp.sum(proportions) == 1,  # Total allocation must sum to 1 - fully invested portfolio
    proportions >= 0           # No short selling
]

## Example 1 - Minimize Risk ###

In [23]:
# Define the optimization problem - minimize risk
objective = cp.Minimize(cp.sum(cp.multiply(standard_deviations, proportions)))
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Check if the problem was solved successfully
if problem.status in ["optimal", "optimal_inaccurate"]:
    optimal_allocation = proportions.value
    optimal_risk = np.sum(standard_deviations * optimal_allocation)
else:
    optimal_allocation = None
    optimal_risk = None

# Generate a grid of feasible allocations
grid_points = 50
allocations = np.linspace(0, 1, grid_points)
solution_space = []

for x in allocations:
    for y in allocations:
        z = 1 - x - y
        if z >= 0:  # Ensure the constraint x + y + z = 1 holds
            risk = np.sum(standard_deviations * np.array([x, y, z]))
            solution_space.append((x, y, z, risk))

# Convert solution space to numpy array
solution_space = np.array(solution_space)

# Separate x, y, z, and risk values
x_vals, y_vals, z_vals, risks = solution_space.T

# Plot the solution space dynamically using Plotly
fig = go.Figure()

# Add the solution space as a scatter plot
fig.add_trace(go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=3, color=risks, colorscale='Reds', colorbar=dict(title='Risk')),  # Risk metric
    showlegend=False,  # Hide from legend
    hovertemplate=(
        '<b>Asset A:</b> %{x:.2f}<br>'
        '<b>Asset B:</b> %{y:.2f}<br>'
        '<b>Asset C:</b> %{z:.2f}<br>'
        '<b>Risk:</b> %{marker.color:.2f}<br>'
        '<extra></extra>'
    )
))

# Add the initial allocation
fig.add_trace(go.Scatter3d(
    x=[initial_proportions[0]],
    y=[initial_proportions[1]],
    z=[initial_proportions[2]],
    mode='markers',
    marker=dict(size=8, color='blue'),
    name='Initial Allocation',
    hovertemplate=f'<b>Asset A:</b> {initial_proportions[0]:.2f}<br>' +
                  f'<b>Asset B:</b> {initial_proportions[1]:.2f}<br>' +
                  f'<b>Asset C:</b> {initial_proportions[2]:.2f}<br>' +
                  f'<b>Risk:</b> {initial_risk:.2f}<br>'
))

# Add the optimal allocation
fig.add_trace(go.Scatter3d(
    x=[optimal_allocation[0]],
    y=[optimal_allocation[1]],
    z=[optimal_allocation[2]],
    mode='markers',
    marker=dict(size=8, color='black'),
    name='Optimal Allocation',
    hovertemplate=f'<b>Asset A:</b> {optimal_allocation[0]:.2f}<br>' +
                  f'<b>Asset B:</b> {optimal_allocation[1]:.2f}<br>' +
                  f'<b>Asset C:</b> {optimal_allocation[2]:.2f}<br>' +
                  f'<b>Risk:</b> {optimal_risk:.2f}<br>'
))

# Add titles and labels
fig.update_layout(
    scene=dict(
        xaxis_title='Asset A',
        yaxis_title='Asset B',
        zaxis_title='Asset C'
    ),
    showlegend=True,
    legend=dict(
        x=1,  # Position the legend to the right
        y=1,  # Position the legend at the top
        xanchor='right',  # Anchor the legend to the right
        yanchor='top'     # Anchor the legend to the top
    )
)

# Show the plot
fig.show()

# Save the figure as an HTML file
pyo.plot(fig, filename='portfolio_allocation_min_risk_1.html', auto_open=True)


'portfolio_allocation_min_risk_1.html'

### Example 2 - Minimize Risk and Maximize ESG ###

In [20]:
# Generate a grid of feasible allocations
grid_points = 50
allocations = np.linspace(0, 1, grid_points)
solution_space = []

for x in allocations:
    for y in allocations:
        z = 1 - x - y
        if z >= 0:  # Ensure the constraint x + y + z = 1 holds
            risk = np.sum(standard_deviations * np.array([x, y, z]))
            esg = np.sum(esg_scores * np.array([x, y, z]))
            solution_space.append((x, y, z, risk, esg))

# Convert solution space to numpy array
solution_space = np.array(solution_space)
x_vals, y_vals, z_vals, risks, esg_vals = solution_space.T

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div(
    style={'backgroundColor': 'white', 'padding': '20px', 'fontFamily': 'Arial'},
    children=[
        html.H1("Interactive Portfolio Optimization", style={'textAlign': 'center', 'color': 'black'}),
        html.Div([
            html.Label("Weight for Risk Minimization (w1):", style={'fontSize': '16px'}),
            dcc.Slider(id='w1-slider', min=0, max=5, step=0.5, value=1,
                       marks={i: str(i) for i in range(0, 6)}, tooltip={"placement": "bottom", "always_visible": True}),
            html.Br(),
            html.Label("Weight for ESG Maximization (w2):", style={'fontSize': '16px'}),
            dcc.Slider(id='w2-slider', min=0, max=5, step=0.5, value=2,
                       marks={i: str(i) for i in range(0, 6)}, tooltip={"placement": "bottom", "always_visible": True}),
        ], style={'margin': '20px 0'}),
        html.Div([
            dcc.Graph(id='risk-chart', style={'display': 'inline-block', 'width': '49%', 'vertical-align': 'top'}),
            dcc.Graph(id='esg-chart', style={'display': 'inline-block', 'width': '49%', 'vertical-align': 'top'}),
        ], style={'display': 'flex', 'justify-content': 'space-between'})
    ]
)


@app.callback(
    [Output('risk-chart', 'figure'),
     Output('esg-chart', 'figure')],
    [Input('w1-slider', 'value'),
     Input('w2-slider', 'value')]
)
def update_charts(w1, w2):
    # Variables
    proportions = cp.Variable(len(assets))

    # Define constraints
    constraints = [
        cp.sum(proportions) == 1,  # Total allocation must sum to 1
        proportions >= 0           # No short selling
    ]

    # Define the optimization problem
    objective = cp.Minimize(
        w1 * cp.sum(cp.multiply(standard_deviations, proportions)) - w2 * cp.sum(cp.multiply(esg_scores, proportions))
    )
    problem = cp.Problem(objective, constraints)

    # Solve the problem
    problem.solve()

    # Check if the problem was solved successfully
    if problem.status in ["optimal", "optimal_inaccurate"]:
        optimal_allocation = proportions.value
        optimal_risk = np.sum(standard_deviations * optimal_allocation)
        optimal_esg = np.sum(esg_scores * optimal_allocation)
    else:
        optimal_allocation = [0, 0, 0]
        optimal_risk = 0
        optimal_esg = 0

    # Create the risk 3D chart
    risk_fig = go.Figure()
    risk_fig.add_trace(go.Scatter3d(
        x=x_vals,
        y=y_vals,
        z=z_vals,
        mode='markers',
        marker=dict(size=3, color=risks, colorscale='Reds', colorbar=dict(title='Risk')),
        name='Solution Space',
        showlegend=False,
        hovertemplate='<b>Asset A:</b> %{x:.2f}<br>' +
                      '<b>Asset B:</b> %{y:.2f}<br>' +
                      '<b>Asset C:</b> %{z:.2f}<br>' +
                      '<b>Risk:</b> %{marker.color:.2f}<br><extra></extra>'
    ))
    # Add initial allocation point
    risk_fig.add_trace(go.Scatter3d(
        x=[initial_proportions[0]],
        y=[initial_proportions[1]],
        z=[initial_proportions[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Initial Allocation',
        hovertemplate='<b>Asset A:</b> %{x:.2f}<br>' +
                      '<b>Asset B:</b> %{y:.2f}<br>' +
                      '<b>Asset C:</b> %{z:.2f}<br>' +
                      f'<b>Risk:</b> {initial_risk:.2f}<br>'
    ))
    risk_fig.add_trace(go.Scatter3d(
        x=[optimal_allocation[0]],
        y=[optimal_allocation[1]],
        z=[optimal_allocation[2]],
        mode='markers',
        marker=dict(size=8, color='black'),
        name='Optimal Allocation',
        hovertemplate='<b style="color:white">Asset A:</b> %{x:.2f}<br>' +
                      '<b style="color:white">Asset B:</b> %{y:.2f}<br>' +
                      '<b style="color:white">Asset C:</b> %{z:.2f}<br>' +
                      f'<b style="color:white">Risk:</b> {optimal_risk:.2f}<br>'
    ))
    risk_fig.update_layout(
        scene=dict(
            xaxis_title='Asset A',
            yaxis_title='Asset B',
            zaxis_title='Asset C',
            bgcolor='white'
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            x=-0.2,  # Position the legend further to the left
            y=1.1,   # Position the legend higher up
            xanchor='left',
            yanchor='top'
        )
    )

    # Create the ESG 3D chart
    esg_fig = go.Figure()
    esg_fig.add_trace(go.Scatter3d(
        x=x_vals,
        y=y_vals,
        z=z_vals,
        mode='markers',
        marker=dict(size=3, color=esg_vals, colorscale='Greens', colorbar=dict(title='ESG')),
        name='Solution Space',
        showlegend=False,
        hovertemplate='<b>Asset A:</b> %{x:.2f}<br>' +
                      '<b>Asset B:</b> %{y:.2f}<br>' +
                      '<b>Asset C:</b> %{z:.2f}<br>' +
                      '<b>ESG:</b> %{marker.color:.2f}<br><extra></extra>'
    ))
    
    # Add initial allocation point for ESG chart
    esg_fig.add_trace(go.Scatter3d(
        x=[initial_proportions[0]],
        y=[initial_proportions[1]],
        z=[initial_proportions[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Initial Allocation',
        hovertemplate='<b>Asset A:</b> %{x:.2f}<br>' +
                      '<b>Asset B:</b> %{y:.2f}<br>' +
                      '<b>Asset C:</b> %{z:.2f}<br>' +
                      f'<b>ESG:</b> {initial_esg:.2f}<br>'
    ))
    esg_fig.add_trace(go.Scatter3d(
        x=[optimal_allocation[0]],
        y=[optimal_allocation[1]],
        z=[optimal_allocation[2]],
        mode='markers',
        marker=dict(size=8, color='black'),
        name='Optimal Allocation',
        hovertemplate='<b style="color:white">Asset A:</b> %{x:.2f}<br>' +
                      '<b style="color:white">Asset B:</b> %{y:.2f}<br>' +
                      '<b style="color:white">Asset C:</b> %{z:.2f}<br>' +
                      f'<b style="color:white">ESG:</b> {optimal_esg:.2f}<br>'
    ))
    esg_fig.update_layout(
        scene=dict(
            xaxis_title='Asset A',
            yaxis_title='Asset B',
            zaxis_title='Asset C',
            bgcolor='white'
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=False
    )
    #write html files
    # risk_fig.write_html('portfolio_allocation_min_risk.html', auto_open=True)
    # esg_fig.write_html('portfolio_allocation_min_risk_max_esg.html', auto_open=True)
    return risk_fig, esg_fig


# Call the function to save the app layout
if __name__ == '__main__':
    app.run_server(debug=True)


### Efficient Frontier ###

In [25]:

# Generate a range of weights for the trade-off, explicitly including w1=0 and w1=1
w1_values = np.concatenate(([0], np.linspace(0, 1, 1000), [1]))
w2_values = 1 - w1_values  # Complementary weights

# Prepare to store results
risks = []
esg_scores_optimal = []

# Iterate over w1 and w2 pairs
for w1, w2 in zip(w1_values, w2_values):
    # Variables
    proportions = cp.Variable(len(assets))

    # Define constraints
    constraints = [
        cp.sum(proportions) == 1,  # Total allocation must sum to 1
        proportions >= 0           # No short selling
    ]

    # Define the optimization problem - trade-off between risk and ESG
    objective = cp.Minimize(
        w1 * cp.sum(cp.multiply(standard_deviations, proportions)) -
        w2 * cp.sum(cp.multiply(esg_scores, proportions))
    )
    problem = cp.Problem(objective, constraints)

    # Solve the problem
    problem.solve()

    # Check if the problem was solved successfully
    if problem.status in ["optimal", "optimal_inaccurate"]:
        optimal_allocation = proportions.value
        risks.append(np.sum(standard_deviations * optimal_allocation))
        esg_scores_optimal.append(np.sum(esg_scores * optimal_allocation))

# Convert results to arrays
risks_filtered = np.array(risks)
esg_scores_filtered = np.array(esg_scores_optimal)

# Plot the efficient frontier dynamically using Plotly
fig = go.Figure()

# Add the efficient frontier
fig.add_trace(go.Scatter(
    x=risks_filtered,
    y=esg_scores_filtered,
    mode='lines+markers',
    marker=dict(size=8),
    line=dict(width=2, dash='solid', color='blue'),
    name='Efficient Frontier',
    hovertemplate=(
        '<b>Risk:</b> %{x:.2f}<br>'
        '<b>ESG Score:</b> %{y:.2f}<br>'
        '<b>Weight W1:</b> %{text:.2f}<br>'
        '<b>Weight W2:</b> %{customdata:.2f}<extra></extra>'
    ),
    text=w1_values[:len(risks_filtered)],  # Add w1 values for hover template
    customdata=w2_values[:len(risks_filtered)]  # Add w2 values to the hover template
))

# Add initial allocation point
fig.add_trace(go.Scatter(
    x=[initial_risk],
    y=[initial_esg],
    mode='markers',
    marker=dict(size=12, color='black', symbol='star'),
    name='Initial Allocation',
    hovertemplate=(
        '<b>Initial Risk:</b> %{x:.2f}<br>'
        '<b>Initial ESG Score:</b> %{y:.2f}<br>'
        '<extra></extra>'
    )
))

# Highlight specific points
min_risk_index = np.argmin(risks_filtered)
max_esg_index = np.argmax(esg_scores_filtered)

# Update layout for clarity
fig.update_layout(
    xaxis_title='Portfolio Risk',
    yaxis_title='Portfolio ESG Score',
    showlegend=True,
    xaxis=dict(range=[min(risks_filtered) * 0.9, max(risks_filtered) * 1.1]),
    yaxis=dict(range=[min(esg_scores_filtered) * 0.9, max(esg_scores_filtered) * 1.1])
)

# Show the plot
fig.show()
#write html file
fig.write_html('efficient_frontier.html', auto_open=True)
